<a href="https://colab.research.google.com/github/aleksandartasic/CUDA-paralelnisistemi/blob/main/jun2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%cu

#include <stdio.h>

#define SIZE 16
#define THREADS_PER_BLOCK 2

__global__ void findMinOnDiagonal(int* matrix, int* result)
{
    __shared__ int sharedMin[THREADS_PER_BLOCK];


    int startIndex = threadIdx.x * SIZE/blockDim.x ; //ne radi !!!!! koliko elemenata obradjuje thread svaki                                 //SIZE/blockDim.x je razmak izmedju pocetnih elemenata blokova. start=0*(16/2)=0; end=1*(16/2)=8
///ako ispada da thread 0 radi sa elementom prve vrste sa indeksom 0, 8, thread 1 radi sa elementom sa elementima prve vrste sa indeksima 1,9
                                                                                      //SIZE/blockDim.x je razmak izmedju pocetnih elemenata blokova. start=0*(16/2)=0; end=1*(16/2)=8

    int minVal = matrix[startIndex * SIZE + startIndex];//opseg onoga sto radi jedan thread je size/blockDim.x;
    int i = startIndex; // < endIndex; i++)

    int val = matrix[i * SIZE + i];
    if (val < minVal)
    minVal = val;

    sharedMin[threadIdx.x] = minVal;

    __syncthreads();

    // Reduce the sharedMin array to find the overall minimum
    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1)
    {
        if (threadIdx.x < stride)
        {
            if (sharedMin[threadIdx.x + stride] < sharedMin[threadIdx.x])
                sharedMin[threadIdx.x] = sharedMin[threadIdx.x + stride];
        }
        __syncthreads();
    }

    if (threadIdx.x == 0)
        atomicMin(result, sharedMin[0]);
}

int main()
{
    int matrix[SIZE * SIZE] = {
        5, 2, 3, 4, 2, 9, 6, 7, 3, 6, 8, 1,5, 2, 3,4,
        2, 9, 6, 7,3, 6, 8, 1, 4, 7, 1, 4,4, 7, 1, 4,
        5, 2, 3, 4,2, 9, 6, 7, 3, 6, 8, 1,4, 7, 1, 4,
        3, 6, 8,-1,5, 2, 3, 4, 2, 9, 6, 7, 3, 6, 8, 1,
        2, 9, 6, 7,3, 6, 8, 1, 4, 7, 1, 4,4, 7, 1, 4,
        5, 2, 3, 4,2, 9, 6, 7, 3, 6, 8, 1,4, 7, 1, 4,
        3, 6, 8, 1,5, 2, 3, 4, 2, 9, 6, 7, 3, 6, 8, 1,
        2, 9, 6, 7,3, 6, 8, -41555555, 4, 7, 1, 4,4, 7, 1, 4,
        5, 2, 3, 4,2, 9, 6, -7777, -9223, 6, 8, 1,4, 7, 1, 4,
         5, 2, 3, 4,2, 9, 6, 7, 3, 6, 8, 1,4, 7, 1, 4,
        3, 6, 8, 1,5, 2, 3, 4, 2, 9,-6, 7, 3, 6, 8, 1,
        2, 9, 6, 7,3, 6, 8, 1, 4, 7, 1, 4,4, 7, 1, 4,
        5, 2, 3, 4,2, 9, 6, 7, 3, 6, 8, 1,-444, 7, 1, 4,
        3, 6, 8, 1,5, 2, 3, 4, 2, 9, 6, 7,-311111, -4444446, 8, 1,
        2, 9, 6, 7,3, 6, 8, 1, 4, 7, 1, 4,4, 7,5, -55554,
        5, 2, 3, 4,2, 9, 6, 7, 3, 6, 8, 1,4, 7, 1, 4,

    };

    int* dev_matrix;
    int* dev_result;
    int result;

    cudaMalloc((void**)&dev_matrix, SIZE * SIZE * sizeof(int));
    cudaMalloc((void**)&dev_result, sizeof(int));

    cudaMemcpy(dev_matrix, matrix, SIZE * SIZE * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_result, &matrix[0], sizeof(int), cudaMemcpyHostToDevice);

    findMinOnDiagonal<<<8, THREADS_PER_BLOCK>>>(dev_matrix, dev_result);

    cudaMemcpy(&result, dev_result, sizeof(int), cudaMemcpyDeviceToHost);

    cudaFree(dev_matrix);
    cudaFree(dev_result);

    printf("Minimal element on the main diagonal: %d\n", result);

    return 0;
}





Minimal element on the main diagonal: -9223



In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <iostream>
using namespace std;

#define BLOCK_SIZE 8
//zadatak gde se sabiraju 2 matrice i gde se u vektor smešta prosecna vrednost po koloni matrice
template <typename T>
__host__ void initialize_vector(T*& v, int n);
__host__ void operate_on_GPU(float* A, float* B, float* C, float* v, int n, int m);
__global__ void kernel_sum(float* A, float* B, float* C, int n, int m);
__global__ void kernel_avg(float* C, float* avg, int n, int m, int el_numb);
__host__ bool check_result(float* A, float* B, float* C, float* v, int n, int m);
template <typename T>
__host__ void print_vector(const char* lbl, T* A, int n);

int main(int argc, char** argv)
{
    float *A, *B, *C, *res;
    int n = 221;

    initialize_vector(A, n * n);
    initialize_vector(B, n * n);
    C = (float*) malloc(sizeof(float) * n * n);
    res = (float*) malloc(sizeof(float) * n);

    operate_on_GPU(A, B, C, res, n, n);

    if(check_result(A, B, C, res, n, n))
      cout << n << " Correct!" << endl;
    else
      cout << n << " False!" << endl;

    free(res);
    free(C);
    free(B);
    free(A);

    return 0;
}

template <typename T>
__host__ void initialize_vector(T*& v, int n)
{
    v = (T*) malloc(sizeof(T) * n);
    for(int i = 0; i < n; i++)
      v[i] = T(i + 1);
}

__host__ void operate_on_GPU(float* A, float* B, float* C, float* res, int n, int m)
{
    float* dev_A, *dev_B, *dev_C, *dev_res;
    dim3 grid(min(256, m / BLOCK_SIZE + 1), min(256, n / BLOCK_SIZE + 1)),
        blck(BLOCK_SIZE, BLOCK_SIZE);

    size_t full = sizeof(float) * n * m,
          part = sizeof(float) * n * m;

    cudaError_t err;

    err = cudaMalloc(&dev_A, full);
    if(err)
      cout << "1A " << cudaGetErrorString(err) << endl;

    err = cudaMalloc(&dev_B, full);
    if(err)
          cout << "1B " << cudaGetErrorString(err) << endl;

    err = cudaMalloc(&dev_C, full);
    if(err)
          cout << "1C " << cudaGetErrorString(err) << endl;

    err = cudaMalloc(&dev_res, part);
    if(err)
          cout << "1res " << cudaGetErrorString(err) << endl;

    err = cudaMemcpy(dev_A, A, full, cudaMemcpyHostToDevice);
    if(err)
          cout << "2A " << cudaGetErrorString(err) << endl;

    err = cudaMemcpy(dev_B, B, full, cudaMemcpyHostToDevice);
    if(err)
          cout << "2B " << cudaGetErrorString(err) << endl;

    kernel_sum<<<grid, blck>>>(dev_A, dev_B, dev_C, n, m);
    kernel_avg<<<grid, blck>>>(dev_C, dev_res, n, m, n);
    int new_n = grid.y;
    grid.y = 1;
    kernel_avg<<<grid, blck>>>(dev_res, dev_res, new_n, m, n);

    err = cudaMemcpy(C, dev_C, full, cudaMemcpyDeviceToHost);
    if(err)
          cout << "4C " << cudaGetErrorString(err) << endl;

    err = cudaMemcpy(res, dev_res, sizeof(float) * m, cudaMemcpyDeviceToHost);
    if(err)
          cout << "4res " << cudaGetErrorString(err) << endl;

    cudaFree(dev_res);
    cudaFree(dev_C);
    cudaFree(dev_B);
    cudaFree(dev_A);
}

__global__ void kernel_sum(float* A, float* B, float* C, int n, int m)
{
    int tid_x = blockIdx.x * blockDim.x + threadIdx.x,
      tid_y = blockIdx.y * blockDim.y + threadIdx.y;

    for(int i = tid_y; i < n; i += gridDim.y * blockDim.y)
      for(int j = tid_x; j < m; j += gridDim.x * blockDim.x)
        C[i * m + j] = A[i * m + j] + B[i * m + j];
}


__global__ void kernel_avg(float* C, float* res, int n, int m, int el_numb)
{
    int tid_x = blockIdx.x * blockDim.x + threadIdx.x;
    int tid_y = blockIdx.y * blockDim.y + threadIdx.y;
    __shared__ float sh[BLOCK_SIZE][BLOCK_SIZE];

    // Initialize shared memory to zero
    sh[threadIdx.y][threadIdx.x] = 0;
    __syncthreads();

    for (int j = tid_x; j < m; j += gridDim.x * blockDim.x)
    {
        for (int i = tid_y; i < n; i += gridDim.y * blockDim.y)
        {
            sh[threadIdx.y][threadIdx.x] += C[i * m + j];
        }

        __syncthreads();

        // Reduce the sum within the block using parallel reduction
        for (int s = blockDim.y >> 1; s > 0; s >>= 1)
        {
            if (threadIdx.y < s)
            {
                sh[threadIdx.y][threadIdx.x] += sh[threadIdx.y + s][threadIdx.x];
            }
            __syncthreads();
        }

        // Store the result in the output array
        if (threadIdx.y == 0)
        {
            if (gridDim.y == 1)
            {
                res[j] = sh[0][threadIdx.x] / (float)el_numb;
            }
            else
            {
                res[blockIdx.y * m + j] = sh[0][threadIdx.x] / (float)el_numb;
            }
        }
        __syncthreads();
    }
}



__host__ bool check_result(float* A, float* B, float* C, float* res, int n, int m)
{
    bool c = true;

    float* tmp = (float*) malloc(sizeof(float) * m);
    for(int j = 0; c && j < m; j++)
    {
      tmp[j] = 0;
      for(int i = 0; i < n; i++)
      {
        c = C[i * m + j] == (A[i * m + j] + B[i * m + j]);
        tmp[j] += C[i * m + j];
      }
    }

    for(int i = 0; c && i < m; i++)
      c = res[i] == (tmp[i] / n);

    free(tmp);
    return c;
}

template <typename T>
__host__ void print_vector(const char* lbl, T* A, int n)
{
    cout << lbl << " = |\t";
    for(int i = 0; i < n; (cout << A[i++] << "\t"));
    cout << "|\n";
}

221 False!



In [ ]:
%%cu
#include <iostream>
#include <cuda_runtime.h>

// CUDA kernel to calculate the minimum along the diagonal
//radi sa razlicitim dimenzijama grida, blokova i sto je najbitnije, radi tako da jedna nit moze da radi sa vise kolona matrice.
__global__ void diagonalMin(int* matrix, int* results, int size) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    int min_val = 10000;

    for (int i=0;i<size/gridDim.x/blockDim.x;i++) { ///samo to radi jedna nit
        int element = matrix[tid * size + tid];
        min_val = min(min_val, element);
        tid += blockDim.x * gridDim.x+1; ///jako bitno da jedna nit zna kako da dodje do narednog elementa.
    }

    // Store the minimum value for this block in shared memory
    __shared__ int block_min[8]; // Assuming a maximum of 256 threads per block
    block_min[threadIdx.x] = min_val;
    __syncthreads();

    // Reduce within the block to find the block minimum
    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (threadIdx.x < stride) {
            block_min[threadIdx.x] = min(block_min[threadIdx.x], block_min[threadIdx.x + stride]);
        }
        __syncthreads();
    }

    // Store the block minimum in the results array
    if (threadIdx.x == 0) {
        results[blockIdx.x] = block_min[0];
    }
}

int main() {
    int size = 16; // Matrix size (assumed square)
    int matrix_size = size * size * sizeof(int);
    //int* matrix = new int[size * size];
    int* dev_matrix;
    int* dev_results;
    int num_blocks =4; // Number of CUDA blocks

    // Allocate memory on the device
    cudaMalloc((void**)&dev_matrix, matrix_size);
    cudaMalloc((void**)&dev_results, num_blocks * sizeof(int));

    // Initialize the matrix on the host

    int matrix[size*size] = {
        5, 2, 3, 4, 2, 9, 6, 7, 3, 6, 8, 1,5, 2, 3,4,
        2, 9, 6, 7,3, 6, 8, 1, 4, 7, 1, 4,4, 7, 1, 4,
        5, 2, -63, 4,2, 9, 6, 7, 3, 6, 8, 1,4, 7, 1, 4,
        3, 6, 8,-55,-6445, 2, 3, 4, 2, 9, 6, 7, 3, 6, 8, 1,
        2, 9, 6, 7,3, 6, 8, 1, 4, 7, 1, 4,4, 7, 1, 4,
        5, 2, 3, 4,2, 9, 6, 7, 3, 6, 8, 1,4, 7, 1, 4,
        3, 6, 8, 1,5, 2, 3, 4, 2, 9, 6, 7, 3, 6, 8, 1,
        2, 9, 6, 7,3, 6, 8, -5, 4, 7, 1, 4,4, 7, 1, 4,
        5, 2, 3, 4,2, 9, 6, -7, 4, 6, 8, 1,4, 7, 1, 4,
         5, 2, 3, 4,2, 9, 6, 7, 3, 6, 8, 1,4, 7, 1, 4,
        3, 6, 8, 1,5, 2, 3, 4, 2, 9,55, 7, 3, 6, 8, 1,
        2, 9, 6, 7,3, 6, 8, 1, 4, 7, 1, 4,4, 7, 1, 4,
        5, 2, 3, 4,2, 9, 6, 7, 3, 6, 8, 1,-4, 7, 1, 4,
        3, 6, 8, 1,5, 2, 3, 4, 2, 9, 6, 7,-3, -4656566, 8, 1,
        2, 9, 6, 7,3, 6, 8, 1, 4, 7, 1, 4,4, 7,-54, -4,
        5, 2, 3, 4,2, 9, 6, 7, 3, 6, 8, 1,4, 7, 1, 4,

    };

    // Copy the matrix from the host to the device
    cudaMemcpy(dev_matrix, matrix, matrix_size, cudaMemcpyHostToDevice);

    // Launch the CUDA kernel
    diagonalMin<<<num_blocks, 2>>>(dev_matrix, dev_results, size);

    // Copy the results back to the host
    int* results = new int[num_blocks];
    cudaMemcpy(results, dev_results, num_blocks * sizeof(int), cudaMemcpyDeviceToHost);

    // Find the minimum among the block results
    int min_diagonal = 100000;
    for (int i = 0; i < num_blocks; i++) {
        min_diagonal = min(min_diagonal, results[i]);
    }

    // Print the minimum diagonal element
    std::cout << "Minimum diagonal element: " << min_diagonal << std::endl;

    // Clean up
    delete[] matrix;
    delete[] results;
    cudaFree(dev_matrix);
    cudaFree(dev_results);

    return 0;
}



Minimum diagonal element: -4656566
munmap_chunk(): invalid pointer



In [ ]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-e4mn85e0
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-e4mn85e0
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=581e17476a30e8b95524b97ad8dbafa97d10c03684ed551d71780c1469e92e8a
  Stored in directory: /tmp/pip-ephem-wheel-cache-x_p02gm2/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out
